In [1]:
!pip install datasets
from datasets import load_dataset, Audio

!pip install transformers
from transformers import pipeline, AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer

!pip install numpy
import numpy as np

!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.1 MB/s eta 0:

In [2]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

In [3]:
sample

{'id': 1326,
 'num_samples': 218880,
 'path': None,
 'audio': {'path': 'dev/1015649832594091373.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0002867 ,
         -0.00020713, -0.0001356 ]),
  'sampling_rate': 16000},
 'transcription': 'in 1989 het hy vir brooks en groening gehelp om the simpsons te skep en hy was verantwoordelik om die eerste skrywerspan vir die program te huur',
 'raw_transcription': 'In 1989 het hy vir Brooks en Groening gehelp om The Simpsons te skep en hy was verantwoordelik om die eerste skrywerspan vir die program te huur.',
 'gender': 0,
 'lang_id': 0,
 'language': 'Afrikaans',
 'lang_group_id': 3}

In [4]:
sample["audio"]["array"]

array([ 0.        ,  0.        ,  0.        , ...,  0.0002867 ,
       -0.00020713, -0.0001356 ])

In [5]:
model_id = "sanchit-gandhi/distilhubert-finetuned-gtzan"
pipe = pipeline("audio-classification", model = model_id)

Some weights of the model checkpoint at sanchit-gandhi/distilhubert-finetuned-gtzan were not used when initializing HubertForSequenceClassification: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
- This IS expected if you are initializing HubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at sanchit-gandhi/distilhubert-finetuned-gtzan and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.wei

In [6]:
def classify_audio(filepath):
    preds = pipe(filepath)
    outputs = {}
    for p in preds:
        outputs[p["label"]] = p["score"]
    return outputs

In [7]:
classify_audio(sample["audio"]["array"])

{'hiphop': 0.6653011441230774,
 'blues': 0.09294696897268295,
 'reggae': 0.0908600464463234,
 'jazz': 0.03263482451438904,
 'disco': 0.02812986820936203}

In [8]:
intf = gr.Interface(fn = classify_audio, inputs = gr.Audio(type = "filepath"), outputs = gr.outputs.Label())

intf.launch(debug = True, share = True)

<ipython-input-8-fb91c64ba7cd>:1: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  intf = gr.Interface(fn = classify_audio, inputs = gr.Audio(type = "filepath"), outputs = gr.outputs.Label())
<ipython-input-8-fb91c64ba7cd>:1: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  intf = gr.Interface(fn = classify_audio, inputs = gr.Audio(type = "filepath"), outputs = gr.outputs.Label())


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://141073555243a3aa52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://141073555243a3aa52.gradio.live
